In [1]:
import sys
!{sys.executable} -m pip install facenet-pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 4.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 4.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 MB 21.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.2 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: pillow 11.1.0
    Uninstalling pillow-11.1.0:
      Successfully uninstalled pillow-11.1.0
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1
    Uninstalling torch-2.5.1:
      Successfully uninstalled torch-2.5.1
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.20.1
    Uninstalling torchvision-0.20.1:
      Successfully uninstalled torchvision-0.20.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastbook 0.

In [9]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
import pandas as pd
import numpy as np
from PIL import Image

In [35]:
import os

# Specify the folder path
folder_path = 'images'

# Get all file names in the folder
file_names = os.listdir(folder_path)

# Optionally, filter to only get files (exclude subdirectories)
file_names = [file for file in file_names if os.path.isfile(os.path.join(folder_path, file))]

# Print the list of file names
print(file_names)

['Aaron_Eckhart_0001.jpg', 'Brad_Gushue_0001.jpg', 'Clare_Latimer_0001.jpg', 'Anthony_Hopkins_0002.jpg', 'Aaron_Patterson_0001.jpg', 'David_Myers_0001.jpg', 'Aaron_Guiel_0001.jpg', 'Aaron_Pena_0001.jpg', 'Aaron_Peirsol_0001.jpg', 'Clint_Lamebear_0001.jpg', 'Brenda_Wilson_0001.jpg', 'Barbara_Walters_0003.jpg', 'Ashton_Kutcher_0001.jpg', 'Billy_Bob_Thornton_0001.jpg', 'Arnold_Schwarzenegger_0003.jpg']


In [36]:
# Initialize MTCNN for face detection and InceptionResnetV1 for feature extraction
mtcnn = MTCNN(image_size=160, margin=0)  # Adjust image_size and margin as needed
resnet = InceptionResnetV1(pretrained='vggface2').eval()

## Extract Feature Vectors from images
- MTCNN is used to detect faces in the image and provide bounding boxes.
- InceptionResnetV1, pre-trained on a large dataset of faces (VGGFace2), is used to extract feature vectors representing the unique characteristics of each detected face.
- These feature vectors can then be used for facial recognition tasks like comparing faces, identifying individuals, and more.

In [67]:
df = pd.DataFrame(columns=['File Name', 'Feature Vector'])
folder_path = 'images'
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    if os.path.isfile(file_path):
        try:
            # Load the image
            img = Image.open(file_path)
            
            # Detect faces in the image
            boxes, _ = mtcnn.detect(img, landmarks=False)
            
            # If no faces are detected, skip this image
            if boxes is None or len(boxes) == 0:
                print(f"No faces detected in {file_name}")
                continue  # Skip to the next image
            
            for box in boxes:
                # Crop the face from the image
                face = img.crop(box.tolist())
                
                # Preprocess the face for InceptionResnetV1
                face_tensor = torch.stack([mtcnn(face)])
                
                # Extract the feature vector
                feature_vector = resnet(face_tensor).detach().numpy()

                # Create a temporary DataFrame to hold the results for the current image
                temp_df = pd.DataFrame({
                    'File Name': [file_name],
                    'Feature Vector': feature_vector.tolist()
                })

                
                # Add the result to the DataFrame
                df = pd.concat([df, temp_df], ignore_index=True)
        except Exception as e:
            print(f"Error processing {file_name}: {e}")
                

In [68]:
df

,File Name,Feature Vector
0,Aaron_Eckhart_0001.jpg,"[-0.002311188727617264, -0.10527217388153076, ..."
1,Brad_Gushue_0001.jpg,"[-0.035913366824388504, 0.04265087470412254, -..."
2,Clare_Latimer_0001.jpg,"[0.030108239501714706, 0.020737478509545326, 0..."
3,Anthony_Hopkins_0002.jpg,"[-0.015597607009112835, 0.03488968685269356, -..."
4,Aaron_Patterson_0001.jpg,"[0.03616906329989433, 0.042094238102436066, 0...."
5,David_Myers_0001.jpg,"[0.05999491736292839, -0.06570962071418762, -0..."
6,Aaron_Guiel_0001.jpg,"[0.045506350696086884, 0.000562878733035177, 0..."
7,Aaron_Pena_0001.jpg,"[0.0493774339556694, -0.008196741342544556, 0...."
8,Aaron_Peirsol_0001.jpg,"[0.0042321570217609406, 0.002834965707734227, ..."
9,Clint_Lamebear_0001.jpg,"[0.023235436528921127, 0.09349582344293594, 0...."


#### We got all 15 images converted to feature vectors

## Differential Privacy
- The Laplace mechanism adds random noise drawn from a Laplace distribution to the feature vector.
- The amount of noise is controlled by the privacy budget (epsilon) and the sensitivity of the data.
- A smaller epsilon value results in more noise and stronger privacy protection.

We need to go through the Dataframe and apply differential privacy on feature vectors which is the privacy preserving technique.

The randomise function of the Laplace mechanism is used to add noise to the feature vector, making it differentially private.

In [45]:
from diffprivlib.mechanisms import Laplace

#### Initialize the Laplace mechanism for differential privacy

In [56]:
epsilon = 1.0  # Privacy budget (epsilon)
mechanism = Laplace(epsilon=epsilon, sensitivity=1)

In [69]:
df['Private Feature Vector'] = df['Feature Vector'].apply(
    lambda feature_vector: [mechanism.randomise(val) for val in np.array(feature_vector)]
)

In [70]:
df

,File Name,Feature Vector,Private Feature Vector
0,Aaron_Eckhart_0001.jpg,"[-0.002311188727617264, -0.10527217388153076, ...","[-0.21113548655704514, 1.3718263810558629, -0...."
1,Brad_Gushue_0001.jpg,"[-0.035913366824388504, 0.04265087470412254, -...","[-0.8876701063542562, 0.1100928728368665, 2.88..."
2,Clare_Latimer_0001.jpg,"[0.030108239501714706, 0.020737478509545326, 0...","[0.6208223562587766, -0.10680521329388501, -0...."
3,Anthony_Hopkins_0002.jpg,"[-0.015597607009112835, 0.03488968685269356, -...","[-0.3088659910182231, -0.5512908165309908, -0...."
4,Aaron_Patterson_0001.jpg,"[0.03616906329989433, 0.042094238102436066, 0....","[-0.60065428368628, 2.528669975895604, 0.59104..."
5,David_Myers_0001.jpg,"[0.05999491736292839, -0.06570962071418762, -0...","[0.24649726641270064, 3.4461946622857638, 1.48..."
6,Aaron_Guiel_0001.jpg,"[0.045506350696086884, 0.000562878733035177, 0...","[1.0570975117926555, 2.750035000864362, 2.1146..."
7,Aaron_Pena_0001.jpg,"[0.0493774339556694, -0.008196741342544556, 0....","[-0.37401100886747274, 0.0695385084096332, 0.1..."
8,Aaron_Peirsol_0001.jpg,"[0.0042321570217609406, 0.002834965707734227, ...","[-0.1597989283252332, 2.164862852860284, 0.116..."
9,Clint_Lamebear_0001.jpg,"[0.023235436528921127, 0.09349582344293594, 0....","[0.4722784242925023, -0.16232059962792966, 1.6..."


### As we can see the new called Private Feature Vector got created which has applied the differential privacy on Fetaure Vector